In [151]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [203]:
from experimentation import data_preparation
from experimentation import regression_OLS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
df1 = data_preparation(file_location = "uk_data.csv", splits = 2, lags = 1)
df2 = regression_OLS(file_location = "uk_data.csv", splits = 2, lags = 1, train_share = 0.8, p_cutoff = 1.00)

In [197]:
df1["split_1"]["split_1_train"]

,gdp,retail_sales,wage_growth,covid_dummy,gdp_lag1,retail_sales_lag1,wage_growth_lag1,covid_dummy_lag1
date,,,,,,,,
2005-02-28,0.000000,-0.614439,-2.110818,0,0.876095,1.244168,1.608579,0.0
2005-03-31,-0.372208,-0.154560,1.347709,0,0.000000,-0.614439,-2.110818,0.0
2005-04-30,1.245330,0.000000,0.797872,0,-0.372208,-0.154560,1.347709,0.0
2005-05-31,-0.369004,-0.619195,0.263852,0,1.245330,0.000000,0.797872,0.0
2005-06-30,0.617284,0.778816,0.000000,0,-0.369004,-0.619195,0.263852,0.0
...,...,...,...,...,...,...,...,...
2011-12-31,0.937866,0.777202,0.000000,0,1.306413,-0.898588,0.219298,0.0
2012-01-31,0.464576,0.128535,0.000000,0,0.937866,0.777202,0.000000,0.0
2012-02-29,-1.040462,-0.128370,0.000000,0,0.464576,0.128535,0.000000,0.0


In [205]:
df2.info

<bound method DataFrame.info of Empty DataFrame
Columns: []
Index: []>